## Select articles: look for hateful comments

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from mongoengine import connect
from hatespeech_models import Tweet, Article

client = connect("hatespeech-selection")
db = client["hatespeech-selection"]

Article.objects.count()

68896

Acá tomamos a partir del 1ro de marzo...

In [2]:
import datetime

start_date = datetime.datetime(2020, 3, 1)

base_query = {
    "dummy__ne": True,
    "comments__19__exists": True,
    "created_at__gte": start_date,
}

Article.objects(**base_query).count()

10314

In [5]:
import random

seeds = [
    # Racismo, Xenofobia
    "China",
    "Cuba", 
    "cubano", 
    "bolivia",
    "paraguayo",
    "judío", 
    "\"camionero\"", 
    # Pobreza, motivos socioeconómicos
    #"policía",
    ("\"presos\"", 100),
    "ladrón", 
    "\"represión\"",
    "criminal", 
    "piqueteros", 
    "\"villas\"", 
    "\"la villa\"", 
    "\"movimientos sociales\"",
    "\"organizaciones sociales\"",
    "\"tomas de tierras\"",
    "\"toma de tierras\"",
    "sindicatos",
    "Guernica",
    "mapuches",
    # Mujeres
    "mamá", 
    "\"de género\"", 
    "\"aborto\"", 
    "\"actriz\"",
    "\"actrices\"",
    "feminista",
    "femicidio",
    "\"enfermera\"",
    "\"madre\"",
    "\"personal doméstico\"",
    "\"empleadas domésticas\"",
    "\"la modelo\"",
    "\"la periodista\"",
    # Saco conductora. Me parece que suma ruido nomás
    ("\"la conductora\"", 50),
    "\"la cantante\"",
    
    # LGBTI
    "travesti",
    "\"trans\"",
    "gay",
    "homosexual",
    
    # Discapacidades?
]

seed_ids = set()
reason = {}
seed_articles = []

queries = {}

for word in seeds:
    limit = None
    if type(word) is tuple:
        word, limit = word
        
    query = list(Article.objects(**base_query).search_text(word))
    
    real_count = len(query)
    random.shuffle(query)
    
    
    nuevos = 0
    queries[word] = query
    
    for art in query:
        if art.id not in seed_ids:
            nuevos += 1
            seed_ids.add(art.id)
            seed_articles.append(art)
            reason[art.id] = f"Article -- {word}"
            
            if limit and nuevos >= limit:
                break
    print(f"{word:<30} ---> {len(seed_articles):<5}, {nuevos:<4} nuevos", end="")
    
    if limit:
        print(f" (limitados a {limit} -- real {real_count })")
    else:
        print("")
    
len(seed_articles)


China                          ---> 352  , 352  nuevos
Cuba                           ---> 418  , 66   nuevos
cubano                         ---> 445  , 27   nuevos
bolivia                        ---> 533  , 88   nuevos
paraguayo                      ---> 594  , 61   nuevos
judío                          ---> 601  , 7    nuevos
"camionero"                    ---> 665  , 64   nuevos
"presos"                       ---> 765  , 100  nuevos (limitados a 100 -- real 175)
ladrón                         ---> 840  , 75   nuevos
"represión"                    ---> 860  , 20   nuevos
criminal                       ---> 895  , 35   nuevos
piqueteros                     ---> 923  , 28   nuevos
"villas"                       ---> 946  , 23   nuevos
"la villa"                     ---> 966  , 20   nuevos
"movimientos sociales"         ---> 988  , 22   nuevos
"organizaciones sociales"      ---> 1014 , 26   nuevos
"tomas de tierras"             ---> 1028 , 14   nuevos
"toma de tierras"              --->

1933

In [6]:
from mongoengine import DoesNotExist
from hatespeech_models import Reply

comment_seeds = [
    # Mujer
    "bija",
    "\"prostituta\"",
    "\"trola\"",
    # LGBTI
    "marica",
    "\"viejo puto\"",    
    "\"trabuco\"",
    "travesti",
    ("\"degenerado\"", 30),
    #"\"puto\"",
    #"\"trolo\"",
    
    # Xenofobia, Racismo, Religión
    
    #"judío",
    "sionista",
    
    # Pobreza
    #"\"negro\"",
    "\"negros de mierda\"",
    "\"negro de mierda\"",
    ("\"villeros\"", 40),
    # Política
    #"\"terrorista\"",
    
    "\"bala\"",
    "\"matarlos\"",
    "\"una bomba\"",
    "\"uno menos\"",
    # Aspecto
    "\"gorda\"",
]

new_article_ids = set()

for word in comment_seeds:
    nuevos = 0
    limit = None
    if type(word) is tuple:
        word, limit = word
    """
    for _ in range(3):
        print("="*80)
        
    print(f"\n\n{word}\n\n")
    """
    for rep in Reply.objects.no_dereference().search_text(word):
        try:
            article = Article.objects(**base_query).get(id=rep.article.id)
            if article.id not in seed_ids:
                nuevos += 1
                seed_ids.add(article.id)
                seed_articles.append(article)
                reason[article.id] = f"Comment -- {word} ({rep.text})"
                if limit and nuevos >= limit:
                    break
            else:
                pass
                #print(f"\nSkipping {article} -- Reason: {reason[article.id]}")
        except DoesNotExist as e:
            pass
            #print(f"\nSkipping {Article.objects.get(id=rep.article.id)} -- {e}")

    print(f"{word:<30} ---> {len(seed_articles):<5}, {nuevos:<4} nuevos", end="")
    
    if limit:
        print(f" (limitados a {limit} -- )")
    else:
        print("")


bija                           ---> 1961 , 28   nuevos
"prostituta"                   ---> 2055 , 94   nuevos
"trola"                        ---> 2149 , 94   nuevos
marica                         ---> 2197 , 48   nuevos
"viejo puto"                   ---> 2246 , 49   nuevos
"trabuco"                      ---> 2252 , 6    nuevos
travesti                       ---> 2278 , 26   nuevos
"degenerado"                   ---> 2308 , 30   nuevos (limitados a 30)
sionista                       ---> 2375 , 67   nuevos
"negros de mierda"             ---> 2419 , 44   nuevos
"negro de mierda"              ---> 2456 , 37   nuevos
"villeros"                     ---> 2496 , 40   nuevos (limitados a 40)
"bala"                         ---> 2827 , 331  nuevos
"matarlos"                     ---> 2869 , 42   nuevos
"una bomba"                    ---> 2971 , 102  nuevos
"uno menos"                    ---> 3058 , 87   nuevos
"gorda"                        ---> 3255 , 197  nuevos


In [16]:
len(seed_ids)

3255

In [20]:
from groups.models import Group

try:
    group = Group.objects.get(name="Seed Articles + Comments")
except DoesNotExist:
    group = Group(name="Seed Articles + Comments")
    
group.articles = seed_articles

group.save()

Seed Articles + Comments group with 3255 tweets
    

In [15]:
from pprint import pprint as pp

for j, art in enumerate(seed_articles):
    if "alberto" not in art.title.lower():
        continue
    print(f"{j:<5} -- {art.title}")
    print("\n")
    pp(art.first_paragraphs)
    print(f"Razón: {reason[art.id]}")
    print("\n\n")

3     -- Alberto F. confia en una "pronta recuperación" basada en el "comercio bilateral con China"


('El presidente Alberto Fernández fue invitado por el gobierno chino a '
 'participar de manera virtual en la apertura de la Feria Internacional de '
 'Comercio de Servicios de China.\n'
 '\n'
 'En este marco, el mandatario fue entrevistado por la agencia china Xinhua, a '
 'la cual expresó que "la participación en esta feria es prueba del compromiso '
 'de mi gobierno con el sector y con la convicción de la importancia de la '
 'economía del conocimiento como multiplicador del bienestar para nuestras '
 'economías".')
Razón: Article -- China



19    -- Alberto Fernández y Xi Jinping acordaron profundizar las relaciones geopolíticas entre Argentina y China


('Alberto Fernández dialogó durante 40 minutos con Xi Jinping para profundizar '
 'las relaciones geopolíticas entre Argentina y China. El Presidente agradeció '
 'el apoyo de Beijing en medio de la pandemia del COVID-19 y su resp

In [9]:
for art_id, r in reason.items():
    if "villeros" in r:
        
        art = Article.objects.get(id=art_id)
        print("="*80)
        print(art)
        print("\n", r)
        print(art.first_paragraphs)
        
        for comm in art.comments[:30]:
            print(comm.text)

1306760417151197184 - clarincom (49 comentarios)
Axel Kicillof intenta sacarle a la ciudad de La Plata un anfiteatro histórico

 Comment -- "villeros" (@clarincom Que ese bobo llegue a gobernador indica una provincia llena de villeros ocupas sucio y cortuptos)
No es que preocupe demasiado la cultura. La obsesión es por el manejo de símbolos que alimenten el marketing político. Desde hace varios días, la Provincia y la Municipalidad de La Plata desarrollan una pelea destemplada por el manejo de la República de los Niños, y ahora también incorporan a esa disputa el histórico Anfiteatro Martín Fierro del Paseo del Bosque, con capacidad para 2.600 butacas al aire libre. Un filón en épocas de confinamiento pandémico.

El aditamento es la decisión de la Comuna local de habilitar las actividades artísticas, bajo protocolo.
@clarincom JJJAJAJJ!!!!! JAJAJAJAJAJAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA!!!!
@clarincom No puede hacer nada por su cuenta el inútil este?
@clarincom Soñá, el Martín Fierro e

In [11]:
distinct_titles = set(art.title for art in seed_articles)

len(distinct_titles)

2487

In [ ]:
for i, art in enumerate(seed_article):
    print("")

Hay notas con igual título (pero son de distintos medios!)

In [12]:
len(seed_ids)

2491

In [61]:
Article.objects.update(set__selected=False)
updated = Article.objects(id__in=seed_ids).update(set__selected=True)
print(f"Marcados como seleccionados {updated} artículos")

Marcados como seleccionados 2489 artículos


## Cuál es la intersección con los que tienen odio?

Busquemos el grupo odioso y veamos cuántas están acá


In [49]:
from groups.models import Group

Group.objects.no_dereference()

[Comments 0.15 group with 4637 tweets
    , Comments 0.20 group with 2577 tweets
    , Comments 0.25 group with 1439 tweets
    , Comments 0.30 group with 762 tweets
    ]

Veamos con el de 0.30

In [53]:
group = Group.objects[3]

In [54]:
group.articles[0].tweet_id

1228308352607846401

In [55]:
in_seeds = sum(art.id in seed_ids for art in group.articles)

print(f"De las notas odiosas, está el {100 * (in_seeds / len(group.articles)):.2f}%")

De las notas odiosas, está el 49.48%


¿Cuáles faltan?

In [56]:
missing = sorted([art for art in group.articles if art.id not in seed_ids], key=lambda a: a.created_at)



In [57]:
for i, art in enumerate(missing):
    print((f"{'=' * 80 }" + '\n') * 3)
    print(art.created_at)
    print(f"{i+1} -- {art}")
    print("\n\n")
    print(art.first_paragraphs)
    #print(art.body[:600])


2020-02-14 13:21:46
1 -- 1228308352607846401 - clarincom (56 comentarios)
Coronavirus: el video con el desesperado mensaje de 14 argentinos que viven en China y reclaman ser repatriados



Los argentinos que se encuentran en la provincia china de Hubei, donde se desató la epidemia del coronavirus​, difundieron un video con un dramático pedido para ser repatriados.

"Somos 14 argentinos que hemos agotado todas las instancias formales de pedido de repatriación. Nuestra situación es difícil, estamos restringidos a salir a la calle. Entre nosotros tenemos 4 niños, el mayor tiene 6 años y el menor apenas 8 meses", dijo Gloria, una de las argentinas que lleva adelante el reclamo, en el video que comenzó a difundirse este viernes.

2020-02-18 12:50:33
2 -- 1229750047211868160 - LANACION (30 comentarios)
Varios argentinos volvieron de China en medio del brote de coronavirus y nadie los controló



En China, la epidemia provocó más de 1800 muertes Crédito: Instagram

En medio del brote por cor

In [49]:
missing[60].comments

[(0.97) @LANACION ¿Por qué mejor no los trasladan a Inglaterra?,
 (0.00) @LANACION Argentina, el aguantadero de america latina.,
 (0.00) @LANACION Que sea a cambio del canal Beagle,
 (0.00) @LANACION Mmm no se si es viable esto...,
 (0.01) @LANACION Están locos???,
 (0.00) @LANACION Y bueno... Si somos el hazmerreír del mundo!!! Lógico,
 (0.17) @LANACION a Malvinas mandenlos....ahi los quieren mucho,
 (0.12) @LANACION Se nos cagan de risa , somos el ajuste social de este vecindario basurero,
 (0.12) @LANACION Vamo.... A pedir ayuda a la reina....a los piratas....acá no mijo!,
 (1.00) @LANACION mmmm, malvinas.. mmm, no creo che,
 (0.00) @LANACION Ahí tienen a su tan amado modelo chileno.,
 (0.03) @LANACION que vivos,
 (0.16) @LANACION Pero están locos,
 (0.99) @LANACION Che, y si le pasamos las coordenadas de Shile a los Ingleses, como los Shilenos le pasaron las nuestras, para que los ayuden como en la guerra de Malvinas, claro lo digo de onda, sin rencor,
 (0.07) @LANACION Ah bueno...

In [122]:
import re

url_regex = re.compile(
r"((?<=[^a-zA-Z0-9])(?:https?\:\/\/|[a-zA-Z0-9]{1,}\.{1}|\b)"
r"(?:\w{1,}\.{1}){1,5}"
r"(?:com|co|org|edu|gov|uk|net|ca|de|es|mil|iq|io|ac|ly|sm){1}"
r"(?:\/[a-zA-Z0-9]{1,})*)"
)



for reply in Reply.objects[:100]:
    if url_regex.search(reply.text):
        print(reply.text)


@perfilcom @Revista_Weekend https://t.co/CnTEp2glse
@perfilcom @Revista_Weekend https://t.co/T5jG94SYob
@infobae Acá nosotros queriendo escapar del 2020 https://t.co/RfKnuEJrxf
@infobae 44 mil a nivel nacional habia hace dos semanas...entonces quiere decir que el ministerio de salud miente? https://t.co/5bsc9JFWHM
@infobae @fernandocarnota A mí el dengue no se me anima jajaja... https://t.co/4969Ss1vub
@infobae El peornismo lo hizo una vez más! https://t.co/jzQt7swwTW
@infobae Alguien vio algún protocolo? Habrá que salir con el traje de Iron Man para evitar picaduras? Saldrá una nueva cuarentena?RT @Libersens3 @PabloperonoS @PatoLiberatore @Mildred_DelRio @alfred0Dmatade2 @LiberalotaCecil @NoOvejita @galledecopas @GuillSvagelj @juanagro29 @Rainier_W0lf https://t.co/EVFGHYTctt
